In [1]:
import os
import torch
import numpy as np
import jiagu
import jieba
import torchtext
import argparse
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import logging
import time
import json
glove_path="/Public/YongkunLiu/2021/weibo-course-design/dataset/sgns.weibo.word"

In [2]:

parser = argparse.ArgumentParser(description='WILDCAT Training')

parser.add_argument("--GLOVE_PATH", default="/Public/YongkunLiu/2021/weibo-course-design/dataset/sgns.weibo.word",help="display a square of a given number", type=str)
parser.add_argument("--NUM_EPOCHS",default=10, help="display a square of a given number", type=int)
parser.add_argument("--BATCH_SIZE",default=256, help="display a square of a given number", type=int)
parser.add_argument("--IMG_SIZE",default=512, help="display a square of a given number", type=int)
parser.add_argument("--LR",default=0.01, help="display a square of a given number", type=float)
parser.add_argument("--WORK_DIR",default='/Public/YongkunLiu/weibo-workdir/', help="display a square of a given number", type=str)
parser.add_argument("--TRAINVAL_PATH",default="./dataset/weibo_senti_100k_trainval_fenci.csv", help="display a square of a given number", type=str)
parser.add_argument("--HIDDEN_SIZE",default=32, help="display a square of a given number", type=int)
parser.add_argument("--NUM_LAYERS",default=2, help="display a square of a given number", type=int)
parser.add_argument("--BIDIRECTIONAL",default="True", help="display a square of a given number", type=str)
parser.add_argument("--RNN_TYPE",default="GRU", help="display a square of a given number", type=str)

#bidirectional
#rnn_type



args = parser.parse_args(args=[])

ctx=torch.device("cuda" if torch.cuda.is_available() else "cpu")
localtime = time.strftime('%Y-%m-%d-%H-%M-%S',time.localtime())

In [3]:
print(time.localtime())

time.struct_time(tm_year=2022, tm_mon=1, tm_mday=9, tm_hour=7, tm_min=50, tm_sec=42, tm_wday=6, tm_yday=9, tm_isdst=0)


In [4]:
def get_logger():
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    formatter = logging.Formatter(fmt="%(message)s")

    sHandler = logging.StreamHandler()
    sHandler.setFormatter(formatter)
    logger.addHandler(sHandler)

    if not os.path.exists(args.WORK_DIR+'/log'):
        os.makedirs(args.WORK_DIR+'/log')
        #LOG_DIR +'/'+ RNN_TYPE+'_'+BIDIRECTIONAL+'_split'+str(SPLIT)+'.log'
    
    fHandler = logging.FileHandler("{}/{}.log".format(args.WORK_DIR+'/log',localtime), mode='w')
    fHandler.setLevel(logging.DEBUG)
    fHandler.setFormatter(formatter)
    logger.addHandler(fHandler)
    return logger
logger=get_logger()
print_json={"time":str(localtime),"epoch":args.NUM_EPOCHS,"BATCH_SIZE":args.BATCH_SIZE,
            "RNN_TYPE":args.RNN_TYPE,"BIDIRECTIONAL":args.BIDIRECTIONAL,"NUM_LAYERS":args.NUM_LAYERS,
           "HIDDEN_SIZE":args.HIDDEN_SIZE}
logger.info(print_json)

{'time': '2022-01-09-01-49-05', 'epoch': 10, 'BATCH_SIZE': 256, 'RNN_TYPE': 'GRU', 'BIDIRECTIONAL': 'True', 'NUM_LAYERS': 2, 'HIDDEN_SIZE': 32}


In [5]:
#读取数据集 1表示积极，0表示消极,要不要清洗[ ] // @ 之类的符号？
file = pd.read_csv("./dataset/weibo_senti_100k_trainval_fenci.csv")
df = pd.DataFrame(file)
# df.loc[0,'review']="haha"
df['review']

0         ["\u6211", "\u4e5f", "\u6562", "\u9732", "\u55...
1         ["\u4e0b\u5348", "\u62ff", "\u5feb\u9012", "\u...
2         ["\u6211", "\u89c9\u5f97", "\u5341\u5927", "\u...
3         ["\u56de\u590d", "@", "\u8d39\u6240\u5efa", "j...
4         ["\u56de\u590d", "@", "?", "\u826f", "?", ":",...
                                ...                        
107984    ["\u56de\u590d", "@", "\u9ed1", "_", "\u9c7c",...
107985    ["\u606d\u795d", "\u5927\u5bb6", "\u9f99\u5e74...
107986    ["[", "\u5fae\u98ce", "]", "\u53bb", "\u6625\u...
107987    ["@", "\u652f\u82f1\u73c9", " ", "\u770b\u770b...
107988    ["\u4e0d\u5149", "\u5077", "\u76f8\u673a", "\u...
Name: review, Length: 107989, dtype: object

In [12]:
# sum=0
# #提前分词，避免CPU瓶颈，将list保存为json字符串格式
# file = pd.read_csv("./dataset/weibo_senti_100k_trainval.csv")
# df = pd.DataFrame(file)
# for idx,i in enumerate(df['review']):
#     text_list=jieba.cut(i, cut_all=False)
#     text_list=list(text_list)
#     text_list=json.dumps(text_list)
    
#     df.loc[idx,'review']=text_list
#     if idx%5000==0:
#         print(idx)

# df.to_csv("./dataset/weibo_senti_100k_trainval_fenci.csv")
# df

In [13]:
text="['hfd','hfdhn']"
t=json.dumps(text)
t=json.loads(t)
t

"['hfd','hfdhn']"

In [14]:
text=df['review'][0]
print(text)
t=json.loads(text)
print(t)

["\u6211", "\u4e5f", "\u6562", "\u9732", "\u554a", "\uff01", "\u95ee\u9898", "\u662f", "\u4eba\u5bb6", "\u8fd9\u8138", "\u591a", "\u5c0f", "\u554a", "\uff01", "[", "\u6293\u72c2", "]"]
['我', '也', '敢', '露', '啊', '！', '问题', '是', '人家', '这脸', '多', '小', '啊', '！', '[', '抓狂', ']']


In [6]:
dataset_names = ['train', 'valid']
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.05, random_state=0)
#StratifiedShuffleSplit 数据集划分函数
#n_splits是train/test的组数，test_size为测试集的比例，andom_state控制是将样本随机打乱
train_split_idx, val_split_idx = next(iter(stratified_split.split(df.review, df.label)))
train_df = df.iloc[train_split_idx].reset_index()
val_df = df.iloc[val_split_idx].reset_index()

In [4]:
def get_train_val_dataset_df(path):
    #读取数据集 1表示积极，0表示消极
    file = pd.read_csv(path)
    df = pd.DataFrame(file)
    dataset_names = ['train', 'valid']
    stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.05, random_state=0)
    #StratifiedShuffleSplit 数据集划分函数
    #n_splits是train/test的组数，test_size为测试集的比例，andom_state控制是将样本随机打乱
    train_split_idx, val_split_idx = next(iter(stratified_split.split(df.review, df.label)))
    train_df = df.iloc[train_split_idx].reset_index()
    val_df = df.iloc[val_split_idx].reset_index()
    return train_df,val_df

In [8]:
train_df,val_df=get_train_val_dataset_df(args.TRAINVAL_PATH)
#一条微博最长为260字，那么为多少词呢...
train_df[:5]


,level_0,Unnamed: 0,Unnamed: 0.1,index,label,review
0,74473,74473,74473,28033,1,"[""\u521a\u6f14"", ""\u5b8c"", ""\u8bdd\u5267"", ""[""..."
1,60729,60729,60729,79237,0,"[""["", ""\u82b1\u5fc3"", ""]"", ""/"", ""/"", ""@"", ""JiA..."
2,46804,46804,46804,106357,0,"[""\u6e90\u4ee3\u7801"", ""\u662f"", ""\u76f8\u58f0..."
3,90949,90949,90949,75928,0,"[""\u4eb2"", ""\u8fd8"", ""\u80fd"", ""\u770b\u5230"",..."
4,52156,52156,52156,15421,1,"[""?"", ""?"", ""?"", ""?"", ""?"", ""/"", ""/"", ""@"", ""\u62..."


In [5]:
class Dataset(Dataset):
    def __init__(self,labels_df,maxlen,transform=None):
        super().__init__()
        self.labels_df = labels_df
        self.transform = transform
        self.maxlen=maxlen

    def __len__(self):
        return self.labels_df.shape[0]


    def __getitem__(self, idx):
        review=self.labels_df.review[idx]
        review=text_pipeline(review,self.maxlen)
        label=self.labels_df.label[idx]
        return review,label
        
        
        # image_name = config.PATH_IMG+'/'+self.labels_df.id[idx]+'.jpg'
        # img = Image.open(image_name)
        # label = self.labels_df.label_idx[idx]
        # if self.transform:
        #     img = self.transform(img)
        # return img, label
train_dataset=Dataset(train_df,100)
val_dataset=Dataset(val_df,100)

NameError: name 'train_df' is not defined

In [6]:
with open("./dataset/glove.json",'r',encoding='utf-8') as f:
    glove_dict=json.load(f)
    

In [20]:
#glove中文词向量,维度为300,转为json
# with open(glove_path,'r',encoding='utf-8') as f:
#     a=f.readlines()
# glove_dict={}
# for word in a[1:]:
#     glove_dict[word.split(' ')[0]]=[float(j) for j in word.split(' \n')[0].split(' ')[1:]]
# with open("./dataset/glove.json",'w') as f:
#     json.dump(glove_dict,f)

In [21]:
#list insert
t=[1,2,3,4,5]
t.insert(2,66)
t

[1, 2, 66, 3, 4, 5]

In [22]:
np.zeros((8, 300))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
#list insert
t=[1,8,3,4,5]
t2=np.array(t)
# t.remove(2)
t2

array([1, 8, 3, 4, 5])

In [7]:
#将句子分词，并且转化为词向量的模块，maxlen为一个句子的长度
#要不要提前分好词，算了...嫌麻烦，先用着
#有了这个模块，似乎就不需要词向量嵌入层nn.Embedding层了？
def text_pipeline(text,maxlen):
    # text_list=jieba.cut(text, cut_all=False)
    # text_list=list(text_list)
    # print(text_list)
    #将glove字典中找不到的词语分为若干个字
    text_list=json.loads(text)
    print(text_list)
    while 1:
        flag=0
        for idx,i in enumerate(text_list):
            #分为最小，若最小也在glove中无，则删除
            if i not in glove_dict:
                # print(i)
                text_list.remove(i)
                for j in i[::-1]:
                    if j in glove_dict:
                        text_list.insert(idx,j)
                flag=1
                break
        if flag==0:
            break
            
    d=maxlen-len(text_list)
    # print(text_list)
    if d<=0:
        text_list=text_list[:maxlen]
    # else:
    #     for i in range(d):
    #         text_list.append("。")
            
    #构建array，将glove词向量拼接成为(100,300)形状
    res=np.zeros((maxlen, 300))
    # print(text_list)
    for idx,i in enumerate(text_list):
        res[idx]=np.array(glove_dict[i])
    
    # text_list=[torch.tensor(glove_dict[i], dtype=torch.float64) for i in text_list]#torch.tensor(glove_dict[i], dtype=torch.float64)
    res=torch.tensor(res,dtype=torch.float32)
    
    return res

In [12]:
print("我今天好开心，要答辩了")
test_pipeline("我今天好开心，要答辩了",10)

我今天好开心，要答辩了


NameError: name 'test_pipeline' is not defined

In [2]:
# text_pipeline("我来到了北京大学刚   演",10)
# print(text_pipeline("我来到了北京大学刚   演",100).shape)
print(train_df['review'][0])
text_pipeline(train_df['review'][0],10)

NameError: name 'train_df' is not defined

In [26]:
list1=[0 for i in range(280)]
list1[5]

0

In [27]:

for idx,i in enumerate(train_dataset):
    print(len(i[0]))
    break
# list

100


In [28]:
# for idx,i in enumerate(list1):
#     print(idx,i)

In [29]:
# #将数据集分为trainval和test，占比分别为0.9和0.1，仅仅运行一次
# file = pd.read_csv("./dataset/weibo_senti_100k_trainval.csv")
# df = pd.DataFrame(file)
# df
# dataset_names = ['train', 'valid']
# stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
# #StratifiedShuffleSplit 数据集划分函数
# #n_splits是train/test的组数，test_size为测试集的比例，andom_state控制是将样本随机打乱
# train_split_idx, val_split_idx = next(iter(stratified_split.split(df.review, df.label)))
# train_df = df.iloc[train_split_idx].reset_index()
# val_df = df.iloc[val_split_idx].reset_index()
# train_df.to_csv("./dataset/weibo_senti_100k_trainval.csv")
# val_df.to_csv("./dataset/weibo_senti_100k_test.csv")

In [30]:
#分词

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))

Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model cost 0.432 seconds.
Loading model cost 0.432 seconds.
Prefix dict has been built successfully.
Prefix dict has been built successfully.


Default Mode: 我/ 来到/ 北京/ 清华大学


In [31]:
print(len(glove_dict['住口']))
a=torch.tensor(glove_dict['住口'], dtype=torch.float64)


300


In [32]:
# for i in train_dataset:
#     print(i)

In [14]:
train_iter =DataLoader(train_dataset,batch_size=args.BATCH_SIZE,shuffle=False,num_workers=1,drop_last=True)
val_iter =DataLoader(val_dataset,batch_size=args.BATCH_SIZE,shuffle=False,num_workers=1,drop_last=True)

In [34]:
# for idx,i in enumerate(train_iter):
#     if idx%100==0:
#         print(idx)

In [8]:
class RNN(nn.Module):
    def __init__(self, input_size=300, hidden_size=32, output_size=2,num_layers=2,bidirectional='True',rnn_type='GRU'):#输入，隐藏层单元，输出，隐藏层数，双向
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        if(bidirectional=='True'):
            self.bidirectional=True
        elif(bidirectional=='False'):
            self.bidirectional=False
        if(rnn_type=='GRU'):
            self.rnn=nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,bidirectional=self.bidirectional)
        elif(rnn_type=='LSTM'):
            self.rnn=nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,bidirectional=self.bidirectional)
        elif(rnn_type=='RNN'):
            self.rnn=nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,bidirectional=self.bidirectional)
        
        if(bidirectional=='True'):
            self.out = nn.Linear(4*hidden_size, output_size)
        else:
            self.out = nn.Linear(1*hidden_size, output_size)

    def forward(self, input, hidden):
        #print(input.size())
        #10*64*8
        output1,hidden=self.rnn(input,hidden)
        #print("output1:{}".format(output1.size()))
        #10*64*32
        if(self.bidirectional==True):
            output2 = torch.cat((output1[0], output1[-1]), -1)
        else:
            output2 = output1[-1]
        #print("output2:{}".format(output2.size()))
        #64*64
        #print("hidden:{}".format(hidden.size()))
        #层数*64*结点数
        #2*64*32
        output=self.out(output2)
        #print(output.size())
        #64*4
        #output=F.softmax(output,dim=0)
        return output, hidden


In [15]:
model=RNN(hidden_size=args.HIDDEN_SIZE,num_layers=args.NUM_LAYERS,bidirectional=args.BIDIRECTIONAL,rnn_type=args.RNN_TYPE)
model.to(ctx)
print(model)

RNN(
  (rnn): GRU(300, 32, num_layers=2, bidirectional=True)
  (out): Linear(in_features=128, out_features=2, bias=True)
)


In [16]:
criterion = nn.CrossEntropyLoss()

In [9]:

def evaluate_loss(data_iter, net,hidden,device, epoch):
    net.eval()
    l_sum, n,val_pred_sum= 0.0, 0,0
    with torch.no_grad():
        for bidx,(X, y) in enumerate(data_iter):
            X=X.float()
            X=X.permute(1,0,2)#[32, 10, 8]->[10,32,8]
            y=y.to(device)
            X=X.to(device)
            
            y_hat,hidden= net(X,hidden)
            loss = criterion(y_hat, y)
            
            l_sum+=loss
            #l_sum +=criterion(y_hat, y).item()
            n += X.size()[1]
            pred=y_hat.max(1, keepdim=True)[1].view(-1)
            pred_sum=pred.eq(y.view_as(pred)).sum().item()
            val_pred_sum+=pred_sum
    print_json={"epoch":str(epoch),"trainval":"val","loss":str(float(l_sum/(bidx+1))),"acc":val_pred_sum/n}
    logger.info(print_json)
    #print('val,epoch:{},pred:{},loss:{}'.format(epoch,val_pred_sum/n,l_sum/n))
    # logger.info("[epoch {}][{}][end] val_loss={:.5f},val_acc:{:.5f}({}/{})".format\
    # (epoch,'val',l_sum/(bidx+1),val_pred_sum/n,int(val_pred_sum),n))
    return l_sum / (bidx+1),val_pred_sum/n

In [18]:
def train(net,train_iter,val_iter,num_epoch,lr_period,lr_decay):
    optimizer = torch.optim.Adam(net.parameters(),lr=args.LR)
    hidden=None
    Max_Acc=0.0
    Min_loss=9999999.9
    for epoch in range(num_epoch):
        net.train()
        n,train_l_sum,train_pred_sum=0,0,0
        if epoch > 0 and epoch % lr_period == 0:
            optimizer.param_groups[0]['lr']=optimizer.param_groups[0]['lr']*lr_decay
        for bidx,(X,Y) in enumerate(train_iter):
            # print(bidx)
            X=X.float()
            X=X.permute(1,0,2)
            X=X.to(ctx)
            Y=Y.to(ctx)
            if(hidden is not None):
                if isinstance (hidden, tuple): # LSTM, state:(h, c)  
                    hidden[0].to(ctx)
                    hidden[1].to(ctx)
                    hidden = (hidden[0].detach(), hidden[1].detach())
                else:   
                    hidden.to(ctx)
                    hidden = hidden.detach()
            
            optimizer.zero_grad()
            #print(str(bidx)+'---------------')
            #print(hidden)
            # print(X,hidden)
            y_hat,hidden= net(X,hidden)
            #print(y_hat.size())
            loss = criterion(y_hat, Y)
            #print(y_hat)
            #print(y)
            #print(loss.item())
            loss.backward()
            optimizer.step()
            #print(loss)
            pred=y_hat.max(1, keepdim=True)[1].view(-1)
            # print(pred)
            pred_sum=pred.eq(Y.view_as(pred)).sum().item()
            #print(pred_sum)
            train_l_sum+=loss
            train_pred_sum+=pred_sum
            n+=X.size(1)
            print_json={"epoch":str(epoch),"batch":str(bidx),"trainval":"train","loss":str(float(loss)),"acc":pred_sum/Y.size()[0]}
            logger.info(print_json)

        if not os.path.exists(args.WORK_DIR+'params'):
            # print(args.WORK_DIR+'params')
            os.makedirs(args.WORK_DIR+'params')
            #{}
        print_json={"epoch":str(epoch),"batch":"end","trainval":"train","loss":str(float(train_l_sum/(bidx+1))),"acc":train_pred_sum/n}
        logger.info(print_json)
        valid_loss,valid_acc=evaluate_loss(val_iter, net,hidden,ctx,epoch)
        if(valid_acc>Max_Acc):
            Max_Acc=valid_acc
            model_best=net
            print_json={"epoch":str(epoch),"save":"save"}
            logger.info(print_json)
            torch.save(net,args.WORK_DIR+'params/best_{}.pth'.format(localtime))
        # logger.info("[epoch {}][{}][end] train_loss={:.5f},loss_classfication={:.5f},loss_confidence={:.5f},train_acc={:.5f}({}/{})".format\
        #             (epoch,'train',train_l_sum/(bidx+1),loss_classfication_sum/(bidx+1),loss_confidence_sum/(bidx+1),train_pred_sum/n,int(train_pred_sum),n))
        
            

In [40]:
train(model,train_iter,val_iter,args.NUM_EPOCHS,2,0.5)

{'epoch': '0', 'batch': '0', 'trainval': 'train', 'loss': '0.6897450685501099', 'acc': 0.5234375}
{'epoch': '0', 'batch': '1', 'trainval': 'train', 'loss': '0.8382019400596619', 'acc': 0.51171875}
{'epoch': '0', 'batch': '2', 'trainval': 'train', 'loss': '0.7035603523254395', 'acc': 0.48828125}
{'epoch': '0', 'batch': '3', 'trainval': 'train', 'loss': '0.6877555251121521', 'acc': 0.4921875}
{'epoch': '0', 'batch': '4', 'trainval': 'train', 'loss': '0.6792736053466797', 'acc': 0.54296875}
{'epoch': '0', 'batch': '5', 'trainval': 'train', 'loss': '0.6725870370864868', 'acc': 0.60546875}
{'epoch': '0', 'batch': '6', 'trainval': 'train', 'loss': '0.668232798576355', 'acc': 0.61328125}


KeyboardInterrupt: 

In [ ]:
# hidden=None
# for idx,(X,Y) in enumerate(train_iter):
#     print(Y)
#     print(X.shape)
#     X=X.to(ctx)
#     outputs=model(X,hidden)
#     print(outputs)
#     break

In [10]:
#将句子分词，并且转化为词向量的模块，maxlen为一个句子的长度
#要不要提前分好词，算了...嫌麻烦，先用着
#有了这个模块，似乎就不需要词向量嵌入层nn.Embedding层了？
def test_pipeline(text,maxlen):
    text_list=jieba.cut(text, cut_all=False)
    text_list=list(text_list)
    print(text_list)
    #将glove字典中找不到的词语分为若干个字
    # text_list=json.loads(text)
    print(text_list)
    while 1:
        flag=0
        for idx,i in enumerate(text_list):
            #分为最小，若最小也在glove中无，则删除
            if i not in glove_dict:
                # print(i)
                text_list.remove(i)
                for j in i[::-1]:
                    if j in glove_dict:
                        text_list.insert(idx,j)
                flag=1
                break
        if flag==0:
            break
            
    d=maxlen-len(text_list)
    # print(text_list)
    if d<=0:
        text_list=text_list[:maxlen]
    # else:
    #     for i in range(d):
    #         text_list.append("。")
            
    #构建array，将glove词向量拼接成为(100,300)形状
    res=np.zeros((maxlen, 300))
    # print(text_list)
    for idx,i in enumerate(text_list):
        res[idx]=np.array(glove_dict[i])
    
    # text_list=[torch.tensor(glove_dict[i], dtype=torch.float64) for i in text_list]#torch.tensor(glove_dict[i], dtype=torch.float64)
    res=torch.tensor(res,dtype=torch.float32)
    
    return res

In [11]:
print("我今天好开心，要答辩了")
print(test_pipeline("我今天好开心，要答辩了",100).shape)
test_pipeline("我今天好开心，要答辩了",100)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


我今天好开心，要答辩了


Loading model cost 0.458 seconds.
Prefix dict has been built successfully.


['我', '今天', '好开心', '，', '要', '答辩', '了']
['我', '今天', '好开心', '，', '要', '答辩', '了']
torch.Size([100, 300])
['我', '今天', '好开心', '，', '要', '答辩', '了']
['我', '今天', '好开心', '，', '要', '答辩', '了']


tensor([[-0.3808, -0.1406,  0.0367,  ...,  0.2216,  0.2000,  0.1449],
        [ 0.3193, -0.1960,  0.1231,  ..., -0.2689, -0.1319,  0.0397],
        [ 0.1812, -0.1460,  0.0240,  ..., -0.0916, -0.4927,  0.0160],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [35]:
#test
ctx=torch.device("cpu")
content="无语了"
content=test_pipeline(content,100)
print(content.shape)
content=content.unsqueeze(0)
print(content.shape)
content=content.to(ctx)
content=content.permute(1,0,2)
print(content.shape)

['无语', '了']
['无语', '了']
torch.Size([100, 300])
torch.Size([1, 100, 300])
torch.Size([100, 1, 300])


In [36]:
model = torch.load("/Public/YongkunLiu/weibo-workdir/params/best_2022-01-09-05-19-15.pth")
for param in model.parameters():
    param.requires_grad = False
model=model.to(ctx)
pred=model(content,None)[0]
print(pred)
pred=pred.max(1, keepdim=True)[1].view(-1).cpu().numpy().tolist()
pred
# print(model(content,None)[1].shape)
# print(model(content,None)[0].shape)

tensor([[ 0.3536, -0.6084]])


[0]

In [ ]:
4,1,32
100,1,300